In [54]:
import boto3
import pandas as pd
import psycopg2
import json

In [3]:
import configparser
config=configparser.ConfigParser()
config.read_file(open('aws.config'))

In [35]:
KEY=config.get("AWS","KEY")
SECRET=config.get("AWS","SECRET")
DWH_IAM_ROLE_NAME='redhshift-s3-access'

In [21]:
ec2 = boto3.resource('ec2',
                    region_name="ap-south-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                    region_name="ap-south-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

iam = boto3.client('iam',
                    region_name="ap-south-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )


redshiftserverless = boto3.client('redshift-serverless',
                                 region_name="ap-south-1",
                                 aws_access_key_id=KEY,
                                 aws_secret_access_key=SECRET)
                                   

In [32]:
bucket = s3.Bucket("shail-dev-bucket-1")

log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'category_pipe.txt',
 'date2008_pipe.txt',
 'listings_pipe.txt',
 'sales_tab.txt',
 'venue_pipe.txt']

In [36]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
roleArn

'arn:aws:iam::451876914000:role/redhshift-s3-access'

In [42]:
response = redshiftserverless.get_credentials(
    dbName='dev',
    durationSeconds=3600,
    workgroupName='default-workgroup'
)
response

{'dbPassword': 'E6t3n1oi7yjXYQ3vpVpGL5CY2m09iAVqE6tSg7V+s/ORcMAdlpmb+fcEtTLKRF+CE4fiOn78b58xar2k58GAQ7wN6Ok4ldRqOutr3ly+l+2GInE6NOAvJwqAfC0OqNhd8csw5FM+VVi1prFejtvF7t/UXKF78ntfp2RGLNeDWJW+FX7BE7ZhRDuLf2qztPU1gqDJ+sBwFoQ/SyTMH999zILun58Y7LHgqnlmB6aUUX/cx6GiwTzYucQQMyvkh2NzrXIhjGooGiB6l9r1aqdbH6pwOHMM1hrQYHR2eadvWfI/ASUkDWEGksX91tYK8+eGJo+pKvgzSQPFZG6FPq5xpxn+PSgxLEGlachIv+5CF2LEFDbxA4LhcOsJHYGEOFfr2aYw2HFEGMBADhG3GMoBd7ZGbS83bJGMUMwDAxa437f1h0hZzLT95vH9b8QOqCOEhw2+JbKRG7OJfc8lBa2F7WB8JZuZo8yHx5+CHs9D93c5JMpSJnJ+UK24OufG+iQo2irAp9gvkgFe2AwgS6DNH48JyIwBr8Y4R2Uy1qShlWnIhPOaZSW27cAaXX7RGywMT8KvavqRBE3NT+CagJ/4Rc/HIcUajTkS/jZNqMJUj9p/OfvdouMSda14RJBWhQePq7E1cH5NwHYiomuh7jskNm5G3yoaiUwUSWm2wK+X/TDVFuIiKims6h26sGWZMPbGumen+LCzO51tiXmlsB6mNJApP/FDA4WxKeIjGnrxmJ9wu8hT+hJFy8CinardMXXAdrgjPJXpj8ouOLTpKqkARadhjDr15HOAeSezxwgOydBta8NE96Yk9cX4YPlZutMu1NQ5UcQKMC8qr6bSh/y8/S7zXyx/ND2zLIJhJ4LghdnZFwUFm5JH8MJR0JxT7CkgebFH5pXnELGeN8kTVWcfBymeagnV/K+eXcy0wRVZ6FamCFKyZAcVIn/PWACKAbiLR/Kw6ZqUzuAnM6vNZP3Hn/sm

In [87]:
try:
    conn = psycopg2.connect (host='default-workgroup.451876914000.ap-south-1.redshift-serverless.amazonaws.com', dbname='shail-db', user='admin', password='Passw0rd',port=5439)
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
conn.set_session (autocommit=True)


In [88]:
cur = conn.cursor()

In [60]:
cur.execute ("""create table users (
userid integer not null distkey sortkey,
username char (8),
firstname varchar (30),
lastname varchar (30),
city varchar (30),
state char (2),
email varchar (100),
phone char (14),
likesports boolean,
liketheatre boolean,
likeconcerts boolean,
likejazz boolean,
likeclassical boolean,
likeopera boolean,
likerock boolean,
likevegas boolean,
likebroadway boolean,
likemusicals boolean);""")

DuplicateTable: Relation "users" already exists


In [63]:
try:

    cur.execute ("""

create table date (
    dateid smallint not null distkey sortkey,
    caldate date not null,
    day char(3) not null,
    week smallint not null,
    month char(3) not null,
    qtr char(5) not null,
    year smallint not null,
    holiday boolean default('N')
);

create table category(
    catid smallint not null distkey sortkey,
    catgroup varchar(10),
    catname varchar(10),
    catdesc varchar(50)
);

create table venue(
    venueid smallint not null distkey sortkey,
    venuename varchar(100),
    venuecity varchar(30),
    venuestate char(2),
    venueseates integer
);

create table events(
    eventid integer not null distkey,
    venueid smallint not null,
    catid smallint not null,
    dateid smallint not null,
    eventname varchar(200)
    starttime timestamp
);

create table listing(
    listid integer not null distkey,
    sellerid integer not null,
    eventid integer not null,
    dateid smallint not null,
    numtickets smallint,
    priceperticket decimal(8,2),
    totalprice decimal(8,2),
    listtime timestamp
);

create table sales(
    salesid integer not null distkey,
    listid integer not null,
    sellerid integer not null,
    buyerid integer not null,
    eventid integer not null,
    dateid smallint not null,
    qtysold smallint,
    pricepaid decimal(8,2),
    commission decimal(8,2),
    saletime timestamp
);

    """)

except psycopg2.Error as e:
    print("Error: Issue creating table!!!")
    print(e)

Error: Issue creating table!!!
Relation "date" already exists



In [79]:
try:
    cur.execute("""
    copy users from 's3://shail-dev-bucket-1/allusers_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::451876914000:role/redhshift-s3-access'
    delimiter '|'
    region 'ap-south-1'
    """)

except psycopg2.Error as e:
    print("Error loading the data into tables!!!")
    print(e)

In [90]:
try:
    cur.execute("""
    copy events from 's3://shail-dev-bucket-1/allevents_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::451876914000:role/redhshift-s3-access'
    delimiter '|'
    region 'ap-south-1'    
    """)
except psycopg2.Error as e:
    print("Error loading the data into tables!!!")
    print(e)

In [82]:
    cur.execute("""
    copy date from 's3://shail-dev-bucket-1/date2008_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::451876914000:role/redhshift-s3-access'
    delimiter '|'
    region 'ap-south-1'
    """)

    cur.execute("""
    copy sales from 's3://shail-dev-bucket-1/sales_tab.txt'
    credentials 'aws_iam_role=arn:aws:iam::451876914000:role/redhshift-s3-access'
    delimiter '\t'
    region 'ap-south-1'
    """)

    cur.execute("""
    copy lisitngs from 's3://shail-dev-bucket-1/listings_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::451876914000:role/redhshift-s3-access'
    delimiter '|'
    region 'ap-south-1'
    """)

    cur.execute("""
    copy category from 's3://shail-dev-bucket-1/category_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::451876914000:role/redhshift-s3-access'
    delimiter '|'
    region 'ap-south-1'
    """)

    cur.execute("""
    copy venue from 's3://shail-dev-bucket-1/venue_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::451876914000:role/redhshift-s3-access'
    delimiter '|'
    region 'ap-south-1'
    """)

InternalError_: Load into table 'sales' failed.  Check 'sys_load_error_detail' system table for details.


In [91]:
conn.close()